In [1]:
import pandas as pd
from pathlib import Path

df = pd.read_csv("../data_processed/road-america/race1_race_state.csv")

# Feature engineering
df["lap_progress"] = df["lap_number"] / df["lap_number"].max()
df["relative_gap"] = df["gap_to_front_s"].fillna(0)

# Define target (example: next lap time or improved gap)
df["target_next_lap"] = df.groupby("car_id")["lap_time_s"].shift(-1)

# Drop rows without target (last lap for each car)
df = df.dropna(subset=["target_next_lap"])

df.head()

,car_id,lap_number,lap_time_s,cumulative_time_s,median_pace,pit_like,position,gap_to_leader_s,gap_to_front_s,lap_progress,relative_gap,target_next_lap
0,GR86-062-012,0,0.535,0.535,176.965,False,1,0.000,0.000,0.000000,0.000,854.368
1,GR86-002-2,0,23.219,23.219,171.217,False,2,22.684,22.684,0.000000,22.684,900.241
2,GR86-004-78,1,229.763,229.763,171.246,False,1,0.000,0.000,0.000031,0.000,208.027
3,GR86-049-88,1,231.577,231.577,169.915,True,2,1.814,1.814,0.000031,1.814,204.594
4,GR86-037-03,1,843.722,843.722,171.780,True,3,613.959,612.145,0.000031,612.145,235.323


In [2]:
df.to_csv("../data_processed/strategy_features.csv", index=False)

In [4]:
import sys
# ensure local package root is on sys.path so "app" can be imported
sys.path.insert(0, str(Path("..").resolve()))

from app.strategy_engine import predict_next_lap, simulate_pit_effect

race_state_path = Path("..") / "data_processed" / "road-america" / "race1_race_state.csv"
race_state_df = pd.read_csv(race_state_path)

# Pick one car & lap to test
row = race_state_df.iloc[50]  # or filter by car_id & lap_number

lap_number = int(row["lap_number"])
median_pace = float(row["median_pace"])
gap_to_front = float(row["gap_to_front_s"])

print("Context:", lap_number, median_pace, gap_to_front)

print("Predicted next lap:", predict_next_lap(
    lap_number=lap_number,
    median_pace=median_pace,
    gap_to_front=gap_to_front,
    pit_like=row["pit_like"],
    max_laps=race_state_df["lap_number"].max()
))

print("Pit simulation:", simulate_pit_effect(
    lap_number=lap_number,
    median_pace=median_pace,
    gap_to_front=gap_to_front,
    pit_loss_seconds=30,
    max_laps=race_state_df["lap_number"].max()
))


Context: 2 171.908 0.0570000000000163
Predicted next lap: 212.85
Pit simulation: {'lap_number': 2, 'no_pit_next_lap_time': 212.85, 'pit_next_lap_time': 210.69, 'pit_penalty_effect': -2.16}
